In [ ]:
#Part 2 HERE

import numpy as np
#from queue import PriorityQueue
import heapq
import sys
import copy

seenStates = {}
costDict = {"A":1, "B":10, "C":100, "D":1000}
backrow = 5
backroomDict = {"A": 3, "B": 5, "C": 7, "D": 9}

class State:
    def __init__(self, grid, agents, cost):
        self.grid = grid
        self.agents = agents
        self.cost = cost
        self.history = []
    def __lt__(self, other):
        if self.cost < other.cost:
            return True
        return False
                
def gridToStr(grid):
    val = "".join(grid[1])
    for i in range(2, backrow+1):
        val += "".join(grid[i])
    return val
        


def readInput(input, fold = False):
    #Grab the grid
    lines = input.split("\n")
    
    if fold:
        #Account for fold
        tmpA, tmpB = lines[-2], lines[1]
        lines[-2] = "  #D#C#B#A#"
        lines[-1] = "  #D#B#A#C#"
        lines.append(tmpA)
        lines.append(tmpB)
    
    M = len(lines)
    N = len(lines[0])
    grid = []
    agents = []
    for line in lines:
        row = []
        for cnt in range(0, N):
            if cnt<len(line):
                row.append(line[cnt])
            else:
                row.append(" ")
        grid.append(row)    
        
    #Grab the agents
    for i in range(0,M):
        for j in range(0,N):
            char = grid[i][j]
            if char != "." and char != " " and char != "#":
                agents.append((char, (i,j)))
    return grid, agents

def moveFromTo(i,j,newI, newJ, aloc, st, q):
    agent = st.agents[aloc]
    newGrid = copy.deepcopy(st.grid.copy())
    
    newGrid[newI][newJ] = newGrid[i][j]
    newGrid[i][j] = "."
    
    newCost = (abs(newI-i) + abs(newJ-j))*costDict[agent[0]]
    totalCost = newCost + st.cost

    #Avoid repeated grid states which will just add additional cost
    gridHash = gridToStr(newGrid)
    if gridHash in seenStates and seenStates[gridHash] <= totalCost:
        return
    seenStates[gridHash] = totalCost
    if DEBUGB:
        print("Moving {} from {},{} to {},{} with cost {}".format(agent[0],i,j,newI,newJ, newCost))
    
    newAgents = copy.deepcopy(st.agents)
    newAgents[aloc] = (agent[0], (newI,newJ))
   
    newState = State(newGrid, newAgents, totalCost)
    if HISTORY:
        newHistory =  copy.deepcopy(st.history)
        newHistory.append((agent[0], (i,j), (newI,newJ)))
        newState.history = newHistory
        
    heapq.heappush(q,(newState.cost, newState))
    return 


def moveToTheirRoom(i,j, aloc, st, q):
    agent = st.agents[aloc]
    agentType = agent[0]
    room = backroomDict[agentType]
    grid = st.grid
    
    #Check that room doesn't have a non-type robot
    for k in range(2,backrow+1):
        cell = grid[k][room]
        if cell != "." and cell != agentType:
            return
    #Find furthest free room
    loc = -1
    for k in range(backrow,1,-1):
        if grid[k][room] == ".":
            loc = k
            break
    if loc == -1:
        print("Error, could not find a free room even when there should be one")
        return
    
    #Calculate if hallway is clear
    for l in range(min(j, room), 1+max(j,room)):
        if l == j:
            continue
        if grid[i][l] != ".":
            return
    if DEBUGB:
        print("Moving {} to room from {},{} to {},{}".format(agent[0],i,j,loc,room))
    moveFromTo(i,j,loc, room, aloc, st, q)
    
def moveToHallway(i,j, aloc, st, q):
    grid = st.grid
    agent = st.agents[aloc]
    agentType = agent[0]

    #If way out of room is blocked, we can't move into the hallway
    for y in range(i-1, 0, -1):
        if grid[y][j] != ".":
            return
    
    #Going right, ignore areas right outside rooms since those will never be blocked
    for k in range(j+1, len(grid[0])):
        if k == 3 or k==5 or k==7 or k==9:
            continue
        if grid[1][k] != ".":
            break
        if DEBUGB:
            print("Moving {} to hallwayfrom {},{} to {},{}".format(agent[0],i,j,1,k))
        moveFromTo(i,j,1,k, aloc, st, q)
        
    #Going left, ignore areas right outside rooms
    for k in range(j-1, -1,-1):
        if k == 3 or k==5 or k==7 or k==9:
            continue
        if grid[1][k] != ".":
            break
        if DEBUGB:
            print("Moving {} to hallwayfrom {},{} to {},{}".format(agent[0],i,j,1,k))
        moveFromTo(i,j,1,k, aloc, st, q)  
    return
        

def isDone(grid, numAgents):
    for i in range(2,backrow+1):
        if grid[i][3] != "A":
            return False
        if grid[i][5] != "B":
                return False
        if grid[i][7] != "C":
                return False
        if grid[i][9] != "D":
                return False
    return True
    
def inHallway(i,j):
    return i == 1
    
def runOneStep(q):
    if len(q) == 0:
        print("Error, queue is empty, exiting")
        sys.exit(1)
    
    #Pop state:
    node = heapq.heappop(q)
    st = node[1]
    agents = st.agents
    grid = st.grid
    if DEBUG:
        print("Looking at grid: ", gridToStr(st.grid))
        print("Cost accum:", st.cost, "len of heap:", len(q))
    
    
    #Check if state is complete
    if isDone(grid, len(st.agents)):
        return st
    
    #Check neighboring states
    for aloc in range(0,len(agents)):
        agent = agents[aloc]
       
        #print("looking at agent", agent[0])
        loc = agent[1]
        i,j = loc[0], loc[1]
        #print("Investigating agent ", agent)
        if inHallway(i,j):
            moveToTheirRoom(i,j, aloc, st, q)
        else:
            moveToHallway(i,j, aloc, st, q)
        #print("q now has", len(q))
    return None

def runAllSteps(q):
    seenStates = {}
    maxNumSteps = 10000000
    st = None
    for i in range(0,maxNumSteps):
        if i % 10000 == 0:
            print("Perfomed ", i, " calculations")
        
        st = runOneStep(q)
        if st:
            break
    if st:
        print("Found shortest path solution")
        print(st.grid)
        print(st.agents)
        print(st.cost)
        if HISTORY:
            print(st.history)
    else:
        print("Could not find solution, heap size is ", len(q))
DEBUGA = False
DEBUGB = False
HISTORY = True

h= None
grid = None
rGrid, rAgents = readInput(input, fold = True)
print(gridToStr(rGrid))
q = []
st = State(rGrid, rAgents, 0)
heapq.heappush(q,(st.cost, st))
runAllSteps(q)        


In [ ]:
#Part 2 Debug Tests
seenStates = {}
inputA = """#############
#...........#
###B#C#B#D###
  #D#C#B#A#
  #D#B#A#C#
  #A#D#C#A#
  #########"""

inputX = """#############
#AA.D.B.B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#.#C#A#
  #########
"""

inputY = """#############
#AA.D...B.BD#
###B#.#.#.###
  #D#.#C#.#
  #D#.#C#C#
  #A#B#C#A#
  #########"""

def reverseAgentLookup(agents, i, j):
    for cnt in range(0,len(agents)):
        if agents[cnt][1] == (i,j):
            return cnt
    return None
DEBUGB = True
HISTORY = True
qu = []
#rGrid, rAgents = readInput(inputA)
#rGrid, rAgents = readInput(inputX)
rGrid, rAgents = readInput(inputY)
st = State(rGrid, rAgents, 0)
#loc = (2,9)
#loc = (1,6)
loc = (1,8)
aloc = reverseAgentLookup(rAgents,loc[0], loc[1])
#moveToHallway(loc[0],loc[1], aloc, st, qu)
#moveToHallway(loc[0],loc[1], aloc, st, qu)
moveToTheirRoom(loc[0],loc[1], aloc, st, qu)
print(len(qu))



In [ ]:
#PART 1 HERE

import numpy as np
#from queue import PriorityQueue
import heapq
import sys
import copy

seenStates = {}
costDict = {"A":1, "B":10, "C":100, "D":1000}
roomDict = {"A": (3,3), "B": (3,5), "C": (3,7), "D": (3,9) }
backRow = 5

class State:
    def __init__(self, grid, agents, cost):
        self.grid = grid
        self.agents = agents
        self.cost = cost
        self.history = []
    def __lt__(self, other):
        if self.cost < other.cost:
            return True
        return False
                
def gridToStr(grid):
    val = "".join(grid[1])
    val += "".join(grid[2])
    val += "".join(grid[3])
    return val
        
def readInput(input):
    #Grab the grid
    lines = input.split("\n")
    M = len(lines)
    N = len(lines[0])
    grid = []
    agents = []
    for line in lines:
        row = []
        for cnt in range(0, N):
            if cnt<len(line):
                row.append(line[cnt])
            else:
                row.append(" ")
        grid.append(row)
    #Grab the agents
    for i in range(0,M):
        for j in range(0,N):
            char = grid[i][j]
            if char != "." and char != " " and char != "#":
                agents.append((char, (i,j)))
    return grid, agents

def moveFromTo(i,j,newI, newJ, aloc, st, q):
    agent = st.agents[aloc]
    newGrid = copy.deepcopy(st.grid.copy())
    
    newGrid[newI][newJ] = newGrid[i][j]
    newGrid[i][j] = "."
    
    newCost = (abs(newI-i) + abs(newJ-j))*costDict[agent[0]]
    totalCost = newCost + st.cost

    #Avoid repeated grid states which will just add additional cost
    gridHash = gridToStr(newGrid)
    if gridHash in seenStates and seenStates[gridHash] <= totalCost:
        return
    seenStates[gridHash] = totalCost
    if DEBUGB:
        print("Moving {} from {},{} to {},{} with cost {}".format(agent[0],i,j,newI,newJ, newCost))
    
    newAgents = copy.deepcopy(st.agents)
    newAgents[aloc] = (agent[0], (newI,newJ))
   
    newState = State(newGrid, newAgents, totalCost)
    if HISTORY:
        newHistory =  copy.deepcopy(st.history)
        newHistory.append((agent[0], (i,j), (newI,newJ)))
        newState.history = newHistory
        
    heapq.heappush(q,(newState.cost, newState))
    return 


def moveToTheirRoom(i,j, aloc, st, q):
    agent = st.agents[aloc]
    agentType = agent[0]
    room = backroomDict[agentType]
    grid = st.grid
    
    #Check that room entrance is not occupied
    if (grid[room[0]-1][room[1]] != "."):
        return

    #If furthest back location is occupied, ensure it's by a robot of the same type and adjust the room
    if grid[room[0]][room[1]] != ".":
        if (grid[room[0]][room[1]] != agentType):
            return
        room = (room[0]-1, room[1])

    #Calculate number of spaces and if hallway is clear
    cnt = 0
    for l in range(min(j, room[1]), 1+max(j,room[1])):
        if l == j:
            continue
        if grid[i][l] != ".":
            return
        cnt +=1
        
    #print(gridToStr(st.grid))
    if DEBUGB:
        print("Moving {} to room from {},{} to {},{}".format(agent[0],i,j,room[0],room[1]))
    moveFromTo(i,j,room[0], room[1], aloc, st, q)
    
def moveToHallway(i,j, aloc, st, q):
    
    grid = st.grid
    backRoomLoc = 4
    hallwayInd = 1
    agent = st.agents[aloc]
    agentType = agent[0]
    #Check way out of room is blocked
    if i == 3:
        if grid[2][j] != ".":
            return
    
    #Going right, ignore areas right outside rooms since those will never be blocked
    for k in range(j+1, len(grid[0])):
        if k == 3 or k==5 or k==7 or k==9:
            continue
        if grid[1][k] != ".":
            break
        if DEBUGB:
            print("Moving {} to hallwayfrom {},{} to {},{}".format(agent[0],i,j,1,k))
        moveFromTo(i,j,1,k, aloc, st, q)
        
    #Going left, ignore areas right outside rooms
    for k in range(j-1, -1,-1):
        if k == 3 or k==5 or k==7 or k==9:
            continue
        if grid[1][k] != ".":
            break
        if DEBUGB:
            print("Moving {} to hallwayfrom {},{} to {},{}".format(agent[0],i,j,1,k))
        moveFromTo(i,j,1,k, aloc, st, q)  
    return
        

def isDone(grid, numAgents):
    if numAgents == 8:
        if grid[2][3] != "A" or grid[3][3] != "A":
            return False
        if grid[2][5] != "B" or grid[3][5] != "B":
            return False
        if grid[2][7] != "C" or grid[3][7] != "C":
            return False
        if grid[2][9] != "D" or grid[3][9] != "D":
            return False
    #For testing
    if numAgents == 4:
        if grid[2][5] != "B" or grid[3][5] != "B":
            return False
        if grid[2][7] != "C" or grid[3][7] != "C":
            return False
    return True
    
def inHallway(i,j):
    return i == 1
    
def runOneStep(q):
    if len(q) == 0:
        print("Error, queue is empty, exiting")
        sys.exit(1)
    
    #Pop state:
    node = heapq.heappop(q)
    st = node[1]
    agents = st.agents
    grid = st.grid
    if DEBUG:
        print("Looking at grid: ", gridToStr(st.grid))
        print("Cost accum:", st.cost, "len of heap:", len(q))
    
    
    #Check if state is complete
    if isDone(grid, len(st.agents)):
        return st
    
    #Check neighboring states
    for aloc in range(0,len(agents)):
        agent = agents[aloc]
       
        #print("looking at agent", agent[0])
        loc = agent[1]
        i,j = loc[0], loc[1]
        #print("Investigating agent ", agent)
        if inHallway(i,j):
            moveToTheirRoom(i,j, aloc, st, q)
        else:
            moveToHallway(i,j, aloc, st, q)
        #print("q now has", len(q))
    return None

def runAllSteps(q):
    seenStates = {}
    maxNumSteps = 1000000
    st = None
    for i in range(0,maxNumSteps):
        if i % 10000 == 0:
            print("Perfomed ", i, " calculations")
        
        st = runOneStep(q)
        if st:
            break
    if st:
        print("Found shortest path solution")
        print(st.grid)
        print(st.agents)
        print(st.cost)
        if HISTORY:
            print(st.history)
    else:
        print("Could not find solution, heap size is ", len(q))
DEBUGA = False
DEBUGB = False
HISTORY = True

h= None
grid = None
rGrid, rAgents = readInput(input)
q = []
st = State(rGrid, rAgents, 0)
heapq.heappush(q,(st.cost, st))
runAllSteps(q)        


In [ ]:
#Part 1 Debug tests
seenStates = {}
inputA = """#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########"""
inputB = """#############
#...B.......#
###B#C#.#D###
  #A#D#C#A#
  #########"""
inputC = """#############
#...B.......#
###B#.#C#D###
  #A#D#C#A#
  #########"""

inputE = """#############
#.....D.....#
###.#B#C#D###
  #A#B#C#A#
  #########"""

def reverseAgentLookup(agents, i, j):
    for cnt in range(0,len(agents)):
        if agents[cnt][1] == (i,j):
            return cnt
    return None

qu = []
#rGrid, rAgents = readInput(inputA)
#rGrid, rAgents = readInput(inputB)
#rGrid, rAgents = readInput(inputC)
rGrid, rAgents = readInput(inputE)
st = State(rGrid, rAgents, 0)
#loc = (2,7)
#loc = (2,5)
#loc = (3,5)
loc = (2,9)
aloc = reverseAgentLookup(rAgents,loc[0], loc[1])
moveToHallway(loc[0],loc[1], aloc, st, qu)
#moveToHallway(loc[0],loc[1], aloc, st, qu)

#moveToTheirRoom(1,6, aloc, st, qu)

#moveToHallway(2,7, 2, st, qu)
#moveToHallway(2,5, 1, st, qu)
print(len(qu))



In [ ]:
input = """#############
#...........#
###B#C#B#D###
  #A#D#C#A#
  #########"""

In [ ]:
input = """#############
#...........#
###B#C#B#.###
  #.#.#C#.#
  #########"""